In [2]:
import pandas as pd

# Carregar o Dataset
df = pd.read_csv('C:\\Users\\neros\\OneDrive\\Documentos\\GitHub\\Projeto-Individual-Machine-Learning\\data\\raw\\housing_data_CDMX.csv')


df.head()

,property_type,places,lat-lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2
0,apartment,MiguelHidalgo,"23.634501,-102.552788",5500000.0,MXN,5450245.50,289775.66,54.0,54.0,5366.215926,101851.85190
1,house,Iztapalapa,"19.31033,-99.068557",1512000.0,MXN,1498321.97,79661.96,80.0,80.0,995.774500,18900.00000
2,apartment,Tlalpan,"19.279771,-99.234597",926667.0,MXN,918284.00,48822.82,100.0,100.0,488.228200,9266.67000
3,apartment,MiguelHidalgo,"23.634501,-102.552788",6410000.0,MXN,6352013.39,337720.36,135.0,135.0,2501.632296,47481.48148
4,apartment,MiguelHidalgo,"19.432657,-99.177444",4416000.0,MXN,4376051.62,232663.51,87.0,87.0,2674.293218,50758.62069


In [3]:
# remover duplicatas
df.drop_duplicates(inplace=True)

# Mostrar as primeiras linhas do DataFrame
df.head()

,property_type,places,lat-lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2
0,apartment,MiguelHidalgo,"23.634501,-102.552788",5500000.0,MXN,5450245.50,289775.66,54.0,54.0,5366.215926,101851.85190
1,house,Iztapalapa,"19.31033,-99.068557",1512000.0,MXN,1498321.97,79661.96,80.0,80.0,995.774500,18900.00000
2,apartment,Tlalpan,"19.279771,-99.234597",926667.0,MXN,918284.00,48822.82,100.0,100.0,488.228200,9266.67000
3,apartment,MiguelHidalgo,"23.634501,-102.552788",6410000.0,MXN,6352013.39,337720.36,135.0,135.0,2501.632296,47481.48148
4,apartment,MiguelHidalgo,"19.432657,-99.177444",4416000.0,MXN,4376051.62,232663.51,87.0,87.0,2674.293218,50758.62069


In [4]:
# Informações sobre o DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15281 entries, 0 to 18233
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   property_type               15281 non-null  object 
 1   places                      15281 non-null  object 
 2   lat-lon                     15281 non-null  object 
 3   price                       15281 non-null  float64
 4   currency                    15281 non-null  object 
 5   price_aprox_local_currency  15281 non-null  float64
 6   price_aprox_usd             15281 non-null  float64
 7   surface_total_in_m2         15281 non-null  float64
 8   surface_covered_in_m2       15281 non-null  float64
 9   price_usd_per_m2            15281 non-null  float64
 10  price_per_m2                15281 non-null  float64
dtypes: float64(7), object(4)
memory usage: 1.4+ MB


In [5]:
import pandas as pd
import numpy as np

def etl_housing_cdmx(caminho_arquivo: str) -> pd.DataFrame:
    """
    Executa o processo de ETL no dataset housing_data_CDMX.csv.
    Inclui:
        - Importação
        - Padronização e limpeza
        - Correção da coluna 'lat-lon'
        - Separação de latitude e longitude
        - Conversões de tipos
        - Remoção de inconsistências

    Parâmetros:
        caminho_arquivo (str): caminho para o arquivo CSV

    Retorna:
        DataFrame limpo e pronto para análise
    """

    # =============================
    # 1. EXTRACT – Carregar dataset
    # =============================
    df = pd.read_csv(caminho_arquivo)

    # =============================
    # 2. TRANSFORM – Limpeza básica
    # =============================

    # Remover colunas totalmente vazias
    df = df.dropna(axis=1, how='all')

    # Remover espaços dos nomes das colunas
    df.columns = df.columns.str.strip()

    # =============================
    # 3. Padronizar coluna lat-lon
    # =============================

    if 'lat-lon' not in df.columns:
        raise ValueError("A coluna 'lat-lon' não foi encontrada no dataset.")

    # Criar coluna auxiliar para evitar conflitos
    df['coord'] = df['lat-lon'].astype(str)

    # Substituir ":" por "," (dados estavam misturados)
    df['coord'] = df['coord'].str.replace(':', ',', regex=False)

    # Separar as duas partes
    df[['latitude', 'longitude']] = df['coord'].str.split(',', expand=True)

    # Converter para float
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

    # Remover linhas onde a conversão falhou
    df = df.dropna(subset=['latitude', 'longitude'])

    # =============================
    # 4. Conversões adicionais úteis
    # =============================

    # Identificar colunas numéricas que vieram como string
    for col in df.columns:
        if df[col].dtype == "object":
            # Tentar converter automaticamente
            df[col] = pd.to_numeric(df[col], errors='ignore')

    # =============================
    # 5. Remover colunas auxiliares
    # =============================
    df = df.drop(columns=['coord', 'lat-lon'])

    # Resetar índice
    df.reset_index(drop=True, inplace=True)

    return df

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15281 entries, 0 to 18233
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   property_type               15281 non-null  object 
 1   places                      15281 non-null  object 
 2   lat-lon                     15281 non-null  object 
 3   price                       15281 non-null  float64
 4   currency                    15281 non-null  object 
 5   price_aprox_local_currency  15281 non-null  float64
 6   price_aprox_usd             15281 non-null  float64
 7   surface_total_in_m2         15281 non-null  float64
 8   surface_covered_in_m2       15281 non-null  float64
 9   price_usd_per_m2            15281 non-null  float64
 10  price_per_m2                15281 non-null  float64
dtypes: float64(7), object(4)
memory usage: 1.4+ MB


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15281 entries, 0 to 18233
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   property_type               15281 non-null  object 
 1   places                      15281 non-null  object 
 2   lat-lon                     15281 non-null  object 
 3   price                       15281 non-null  float64
 4   currency                    15281 non-null  object 
 5   price_aprox_local_currency  15281 non-null  float64
 6   price_aprox_usd             15281 non-null  float64
 7   surface_total_in_m2         15281 non-null  float64
 8   surface_covered_in_m2       15281 non-null  float64
 9   price_usd_per_m2            15281 non-null  float64
 10  price_per_m2                15281 non-null  float64
dtypes: float64(7), object(4)
memory usage: 1.4+ MB


In [7]:
df.to_csv('C:\\Users\\neros\\OneDrive\\Documentos\\GitHub\\Projeto-Individual-Machine-Learning\\data\\processed\\housing_data_CDMX_cleaned.csv', index=False)

In [11]:
# Copiar a coluna para evitar problemas
df['coord'] = df['lat-lon'].astype(str)

# Padronizar separador: transformar ':' em ','
df['coord'] = df['coord'].str.replace(':', ',', regex=False)

# Agora separar latitude e longitude
df[['latitude', 'longitude']] = df['coord'].str.split(',', expand=True)

# Converter para float
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

# Remover linhas inválidas (onde não converteu)
df = df.dropna(subset=['latitude', 'longitude'])

# Remover colunas auxiliares
df = df.drop(columns=['coord', 'lat-lon'])


In [15]:
df.to_csv('C:\\Users\\neros\\OneDrive\\Documentos\\GitHub\\Projeto-Individual-Machine-Learning\\data\\processed\\housing_data_CDMX_cleaned.csv', index=False)